In [ ]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InR

In [ ]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-07 02:01:54--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.89MB/s    in 0.2s    

2023-03-07 02:01:55 (4.89 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [ ]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [ ]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz" 
#url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1/22-big-data/day_3/user_data.csv"
spark.sparkContext.addFile(url)
cleaned_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
cleaned_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [ ]:
# Get the number of rows in the DataFrame.
cleaned_df.count()


9002021

# Transform the Data

## Create the "review_id_table".

In [ ]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = cleaned_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3W4P9UBGNGH1U|   16414143|B00YL0EKWE|     852431543| 2015-08-31|
|R15V54KBMTQWAY|   50800750|B00XK95RPQ|     516894650| 2015-08-31|
| RY8I449HNXSVF|   15184378|B00SXRXUKO|     984297154| 2015-08-31|
|R18TLJYCKJFLSR|   10203548|B009V5X1CE|     279912704| 2015-08-31|
|R1NK26SWS53B8Q|     488280|B00D93OVF0|     662791300| 2015-08-31|
|R11LOHEDYJALTN|   13334021|B00XVGJMDQ|     421688488| 2015-08-31|
|R3ALQVQB2P9LA7|   27520697|B00KQW1X1C|     554285554| 2015-08-31|
|R3MWLXLNO21PDQ|   48086021|B00IP1MQNK|     488006702| 2015-08-31|
|R2L15IS24CX0LI|   12738196|B00HVORET8|     389677711| 2015-08-31|
|R1DJ8976WPWVZU|   15867807|B00HX3G6J6|     299654876| 2015-08-31|
|R3MRWNNR8CBTB7|    1972249|B00U4NATNQ|     577878727| 2015-08-31|
|R1DS6DKTUXAQK3|   10956619|B00SZEFDH8|     654620704| 2015-08

## Create the "products" Table

In [ ]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = cleaned_df.select(["product_id", "product_title"])
products_df.drop_duplicates
products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00YL0EKWE|LG G4 Case Hard T...|
|B00XK95RPQ|Selfie Stick Fibl...|
|B00SXRXUKO|Tribe AB40 Water ...|
|B009V5X1CE|RAVPower® Element...|
|B00D93OVF0|Fosmon Micro USB ...|
|B00XVGJMDQ|iPhone 6 Case, Vo...|
|B00KQW1X1C|Nokia Lumia 630 R...|
|B00IP1MQNK|Lumsing 10400mah ...|
|B00HVORET8|iPhone 5S Battery...|
|B00HX3G6J6|HTC One M8 Screen...|
|B00U4NATNQ|S6 Case - Bear Mo...|
|B00SZEFDH8|BLU Studio X, Unl...|
|B00JRJUL9U|EZOPower 5-Port U...|
|B00KQ4T0HE|iPhone 6S Case &i...|
|B00M0YWKPM| iPhone 6s Plus Case|
|B00KDZEE68|zBoost ZB575-A TR...|
|B00BJN45GM|OtterBox Defender...|
|B00SA86SXW|Aduro PowerUP 30W...|
|B00Q3I68TU|LilGadgets Connec...|
|B00TN4J1TA|Anker Aluminum Mu...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [ ]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
from pyspark.sql.functions import asc
customers_df = cleaned_df.select(["customer_id"])
customers_df.groupBy("customer_ID").agg({'customer_id':'count'}).show()



+-----------+------------------+
|customer_ID|count(customer_id)|
+-----------+------------------+
|   46909180|                 6|
|   42560427|                 7|
|   43789873|                 3|
|   22037526|                 2|
|   34220092|                 2|
|   42801586|                 1|
|    9565734|                 2|
|   15829398|                 1|
|   38247118|                 1|
|   32478248|                 2|
|   48114630|                 1|
|   23085063|                 1|
|   32787070|                 3|
|   43515569|                 1|
|    4919528|                 2|
|    5088547|                 2|
|   41852407|                 3|
|   49703087|                 1|
|   12713799|                 1|
|   36728141|                 8|
+-----------+------------------+
only showing top 20 rows



## Create the "vine_table".

In [ ]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = cleaned_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|
|R15V54KBMTQWAY|          4|            0|          0|   N|
| RY8I449HNXSVF|          5|            0|          0|   N|
|R18TLJYCKJFLSR|          5|            0|          0|   N|
|R1NK26SWS53B8Q|          5|            0|          0|   N|
|R11LOHEDYJALTN|          5|            0|          0|   N|
|R3ALQVQB2P9LA7|          4|            0|          0|   N|
|R3MWLXLNO21PDQ|          5|            0|          0|   N|
|R2L15IS24CX0LI|          5|            0|          0|   N|
|R1DJ8976WPWVZU|          3|            0|          0|   N|
|R3MRWNNR8CBTB7|          5|            0|          0|   N|
|R1DS6DKTUXAQK3|          5|            0|          0|   N|
| RWJM5E0TWUJD2|          5|            0|          0|   N|
|R1XTJKDYNCRGAC|          1|            

# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)
review_id_df.printSchema()


In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)
products_df.printSchema()


In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)
customers_df.printSchema()


In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)
vine_df.printSchema()
